In [1]:
import numpy as np
import os
import random
import pandas as pd
import datetime
import math
from math import e
import datetime
from scipy.interpolate import interp1d
import time, datetime
from pandas.core.frame import DataFrame
from db_models import GmsMonitor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:

def mongodb_connect():
    import mongoengine
    mongoengine
    mongoengine.connect('nfca_db', host='dgx.server.ustb-ai3d.cn', port=27017, username='nfca', password='nfca')

In [4]:
def queryset2df(query_data):
    """
    queryset转dataframe
    :return:
    """
    dic = {"time":[], "value":[]}
    for q in query_data:
        dic["time"].append(q['time'])
        dic["value"].append(q['Monitoring_value'])
    # 存在数据库多条目同一时间同一值、多条目同一时间不同值的情况，仅保留首条数据
    df = pd.DataFrame(dic).drop_duplicates(subset=['time'])
    return df

In [5]:
def get_time_end(th_id, time_range, unfilter_data):
    time_list = []
    unfilter_data = unfilter_data
    for point_id in point[th_id]:
        time_list.append(
            unfilter_data[point_id].loc[time_range[0]:time_range[1]]['time'][-1])
    return min(time_list)

def get_time_start(th_id, time_range, unfilter_data):
    time_list = []
    unfilter_data = unfilter_data
    for point_id in point[th_id]:
        time_list.append(
            unfilter_data[point_id].loc[time_range[0]:time_range[1]]['time'][0])
    return max(time_list)

In [36]:
def pd_aggregation(data):
    dense_series_1s = data.resample('1S').interpolate("linear")
    series_1m = dense_series_1s.groupby(pd.Grouper(freq='5Min')).aggregate(np.mean)
    return series_1m

In [71]:
point = {
    1: [5, 7, 11, 90, 67, 80],
    "name": ["feed_c", "out_c", "feed_f", "out_f", "pressure", "out_f_setting"]
}
point_set = 69

In [72]:
def gene_data(th_id, time_range=None):
    unfilter_data = {}
    from db_models import GmsMonitor
    if time_range is not None:
        for i in point[th_id]:
            print(i)
            unfilter_data[i] = (
                queryset2df(GmsMonitor.objects(time__gte=time_range[0], time__lt=time_range[1], point_id=i)
                            .only('time', 'Monitoring_value').as_pymongo())
                .set_index(['time'], drop=False)
                .iloc[::-1].sort_index(ascending=True))
    else:
        for i in point[th_id]:
            unfilter_data[i] = (
                queryset2df(GmsMonitor.objects(point_id=i)
                            .only('time', 'Monitoring_value').as_pymongo())
                # .set_index(['time'], drop=False)
                .iloc[::-1].sort_index(ascending=True))
    print("data_collected")
    df_list = []
    for point_id in point[th_id]:
        # print(point_id)
        start_time = get_time_start(th_id, time_range, unfilter_data)
        end_time = get_time_end(th_id, time_range, unfilter_data)
        df_data = unfilter_data[point_id].loc[start_time:end_time]
        # print(df_data)
        df_data = pd_aggregation(df_data)
        df_data = df_data.rename(columns={'value':point_id})
        df_list.append(df_data)
   
    df_merge = df_list[0]
    for i in range(len(df_list) - 1): 
        df_merge = df_merge.merge(df_list[i + 1], on='time')
    df_merge.rename(columns={point[th_id][i]: point['name'][i] for i in range(len(point['name']))},
                          inplace=True)
    df_merge.to_csv('df_thickener_%s.csv' % th_id)
    print("data_generated")
    return df_merge

In [73]:
time_range = (datetime.datetime.now()- datetime.timedelta(hours=72), datetime.datetime.now())
# time_range = (datetime.datetime(2022, 5, 8, 7, 6, 44, 564487), datetime.datetime(2022, 5, 9, 7, 6, 44, 564501))
mongodb_connect()
df_1 = gene_data(1, time_range)
# df_2 = gene_data(2, time_range)
# df = [df_1, df_2]
df = df_1

5
7
11
90
67
80
data_collected
data_generated


In [74]:
df_1

,feed_c,out_c,feed_f,out_f,pressure,out_f_setting
time,,,,,,
2022-05-10 09:25:00,17.653204,61.996134,621.040228,140.389410,48.515069,158.174258
2022-05-10 09:30:00,16.564407,62.970862,561.446895,136.302434,47.084535,169.922788
2022-05-10 09:35:00,17.453896,63.332662,629.499202,143.314894,46.547016,176.436190
2022-05-10 09:40:00,16.157694,61.859938,536.887342,141.766153,47.221003,165.006779
2022-05-10 09:45:00,18.930741,61.194826,705.753476,139.917255,47.562318,153.528768
...,...,...,...,...,...,...
2022-05-12 05:00:00,26.409913,62.168513,543.363551,178.938419,57.138134,161.573522
2022-05-12 05:05:00,31.570429,62.812031,636.109054,176.200961,56.783933,166.685083
2022-05-12 05:10:00,30.635693,62.842136,637.082087,185.224416,56.845052,169.478493


In [75]:
def MSE(obs, set_value):
    return (set_value - obs) ** 2

In [76]:
a = GmsMonitor.objects.filter(
                    point_id=point_set
                ).order_by("-time").limit(1)[0].to_mongo().to_dict()['Monitoring_value']

In [77]:
a

62.0

In [78]:
df_1['reward'] = 0

In [79]:
df_1['reward'] =  - MSE(df_1['out_c'], a)

In [80]:
df_1

,feed_c,out_c,feed_f,out_f,pressure,out_f_setting,reward
time,,,,,,,
2022-05-10 09:25:00,17.653204,61.996134,621.040228,140.389410,48.515069,158.174258,-0.000015
2022-05-10 09:30:00,16.564407,62.970862,561.446895,136.302434,47.084535,169.922788,-0.942573
2022-05-10 09:35:00,17.453896,63.332662,629.499202,143.314894,46.547016,176.436190,-1.775988
2022-05-10 09:40:00,16.157694,61.859938,536.887342,141.766153,47.221003,165.006779,-0.019617
2022-05-10 09:45:00,18.930741,61.194826,705.753476,139.917255,47.562318,153.528768,-0.648306
...,...,...,...,...,...,...,...
2022-05-12 05:00:00,26.409913,62.168513,543.363551,178.938419,57.138134,161.573522,-0.028397
2022-05-12 05:05:00,31.570429,62.812031,636.109054,176.200961,56.783933,166.685083,-0.659394
2022-05-12 05:10:00,30.635693,62.842136,637.082087,185.224416,56.845052,169.478493,-0.709193


In [81]:
df_1.to_csv("df_thickener_1_RL_test.csv", index=False)